In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [3]:
# check mrubash1/Origin for downloading and updating to s3

In [4]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
#ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID', 'reviewerName']

In [5]:
ratings

DataFrame[asin: string, overall: double, reviewerID: string, reviewerName: string]

In [6]:
#numRatings = ratings.count()
#numRatings

In [7]:
users = ratings.map(lambda r: r.reviewerID).distinct().zipWithIndex()
#users.take(5)

In [ ]:
#numUsers = users.count()
#print numUsers

In [ ]:
users = ratings.map(lambda r: r.reviewerID).distinct().zipWithIndex()

In [ ]:
movies = ratings.map(lambda r: r.asin).distinct().zipWithIndex()

In [1]:
movies.take(1)

NameError: name 'movies' is not defined

In [10]:
moviesDf = sqlContext.createDataFrame(movies, ['asin', 'mid'])
print moviesDf.take(5)
print type(movies)

[Row(asin=u'0310242444', mid=5621), Row(asin=u'1561764078', mid=5622), Row(asin=u'0783222521', mid=5623), Row(asin=u'0767821556', mid=5624), Row(asin=u'1580022510', mid=5625)]
<class 'pyspark.rdd.PipelinedRDD'>


In [11]:
usersDf = sqlContext.createDataFrame(users, ['reviewerID', 'uid'])
print usersDf.take(5)

[Row(reviewerID=u'A1IC2MFN9E042G', uid=174995), Row(reviewerID=u'A1OVMU30I1U7KG', uid=174996), Row(reviewerID=u'ANK68QCUYUVVX', uid=174997), Row(reviewerID=u'A3IENYX0TU99B8', uid=174998), Row(reviewerID=u'A2PPC2ABSLPWT2', uid=174999)]


In [12]:
rawRatings1 = ratings.join(usersDf, ratings.reviewerID == usersDf.reviewerID)
rawRatings1.printSchema()


root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- uid: long (nullable = true)



In [13]:
rawRatings2 = rawRatings1.join(moviesDf, ratings.asin == moviesDf.asin)
rawRatings2.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- uid: long (nullable = true)
 |-- asin: string (nullable = true)
 |-- mid: long (nullable = true)



In [14]:
print rawRatings2.take(1)[0].asin

0790732068


In [15]:
rawRatings3 = rawRatings2.map(lambda row: (row.uid, row.mid, row.overall)).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [16]:
rawRatings3.take(10)

[(165835, 9392, 5.0),
 (247012, 9392, 5.0),
 (113594, 9629, 5.0),
 (133281, 7101, 5.0),
 (81151, 6190, 5.0),
 (203099, 7257, 5.0),
 (32361, 9392, 4.0),
 (89157, 4401, 5.0),
 (92394, 9392, 5.0),
 (52274, 9392, 5.0)]

In [17]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
rank = 100
numIterations = 10
model = ALS.train(rawRatings3, rank, numIterations)

In [18]:
#type(model)

In [19]:
#type(ALS)

In [20]:
#userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin').map(lambda row: (row.userID, row.movieID))

In [21]:
#userMoviePair.take(5)
#type(userMoviePair)

In [22]:
#recommendations = model.predictAll(userMoviePair)

In [23]:
#type(recommendations)

In [24]:
#recommendations.take(5)